# Install Packages and Setup Variables


In [1]:
!pip install -q huggingface-hub==0.27.1 llama-index==0.10.49 llama-index-vector-stores-chroma==0.1.9 google-generativeai==0.5.4 llama-index-llms-gemini==0.1.11 openai==1.59.8 chromadb==0.5.5

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 3.3 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 150.7/150.7 kB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 455.6/455.6 kB 16.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 584.3/584.3 kB 33.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 66.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 679.1/679.1 kB 29.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.4/15.4 MB 47.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 278.6/278.6 kB 17.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.8/94.8 kB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 65.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 50.9 MB/s eta 0

In [2]:
import os

# Set the following API Keys in the Python environment. Will be used later.
# os.environ["OPENAI_API_KEY"] = "[OPENAI_API_KEY]"
# os.environ["GOOGLE_API_KEY"] = "<YOUR_API_KEY>"

from google.colab import userdata
os.environ["OPENAI_API_KEY"] = userdata.get('openai_api_key')
os.environ["GOOGLE_API_KEY"] = userdata.get('Google_api_key')

# Load a Model


In [3]:
from llama_index.llms.openai import OpenAI
from llama_index.core import Settings
from llama_index.embeddings.openai import OpenAIEmbedding

Settings.llm = OpenAI(temperature=1, model="gpt-4o-mini")
Settings.embed_model = OpenAIEmbedding(model="text-embedding-3-small")

# Load Indexes


Downloading vector store from Huggingface hub

In [4]:
from huggingface_hub import hf_hub_download
vectorstore = hf_hub_download(repo_id="jaiganesan/ai_tutor_knowledge", filename="vectorstore.zip",repo_type="dataset",local_dir="/content")

vectorstore.zip:   0%|          | 0.00/97.2M [00:00<?, ?B/s]

In [5]:
!unzip /content/vectorstore.zip

Archive:  /content/vectorstore.zip
   creating: ai_tutor_knowledge/
   creating: ai_tutor_knowledge/684af133-f877-4230-bde4-575cf53b6688/
  inflating: ai_tutor_knowledge/684af133-f877-4230-bde4-575cf53b6688/length.bin  
  inflating: ai_tutor_knowledge/684af133-f877-4230-bde4-575cf53b6688/index_metadata.pickle  
  inflating: ai_tutor_knowledge/684af133-f877-4230-bde4-575cf53b6688/link_lists.bin  
  inflating: ai_tutor_knowledge/684af133-f877-4230-bde4-575cf53b6688/header.bin  
  inflating: ai_tutor_knowledge/684af133-f877-4230-bde4-575cf53b6688/data_level0.bin  
  inflating: ai_tutor_knowledge/chroma.sqlite3  


In [6]:
# Load the vector store from the local storage.
import chromadb
from llama_index.vector_stores.chroma import ChromaVectorStore

db2 = chromadb.PersistentClient(path="/content/ai_tutor_knowledge")
chroma_collection = db2.get_or_create_collection("ai_tutor_knowledge")
vector_store = ChromaVectorStore(chroma_collection=chroma_collection)

In [7]:
from llama_index.core import VectorStoreIndex
from llama_index.embeddings.openai import OpenAIEmbedding

# Create the index based on the vector store.
index = VectorStoreIndex.from_vector_store(vector_store=vector_store)

In [8]:
query_engine = index.as_query_engine(similarity_top_k=10)

res = query_engine.query("Explain how Advance RAG works?")

In [9]:
res.response

'Advance Retrieval-Augmented Generation (RAG) systems incorporate various methodologies to enhance the quality and efficiency of contextual information retrieval for language generation tasks. These systems utilize specialized techniques such as Self-Reflective RAG (Self-RAG) and Corrective RAG (CRAG), which focus on improving the relevance and accuracy of retrieved documents. \n\nSpecifically, Advance RAG methods, like SPECULATIVE RAG, aim to optimize processing speed while enhancing reasoning capabilities. They achieve this through a divide-and-conquer strategy, where a smaller specialized language model (the RAG drafter) generates multiple answer drafts from subsets of retrieved documents. These drafts are then verified by a larger generalist language model (the RAG verifier), which evaluates their quality and selects the best candidate for inclusion in the final output.\n\nThis collaborative approach allows for the generation of diverse and high-quality responses while minimizing c

# RankGPT


In [10]:
from llama_index.core.postprocessor.rankGPT_rerank import RankGPTRerank

rankGPT = RankGPTRerank(top_n=3,llm=Settings.llm, verbose=True)

In [11]:
# Define a query engine that is responsible for retrieving related pieces of text,
# and using a LLM to formulate the final answer.
# The `node_postprocessors` function will be applied to the retrieved nodes.
query_engine = index.as_query_engine(similarity_top_k=10, node_postprocessors=[rankGPT])

res = query_engine.query("Explain how Retrieval Augmented Generation (RAG) works?")

After Reranking, new rank list for nodes: [1, 0, 3, 9, 8, 7, 5, 6, 2, 4]

In [12]:
res.response

"Retrieval-Augmented Generation (RAG) combines two main components: Retrieval and Generation. \n\n1. **Retrieval Component**: \n   - This component extracts relevant information from external knowledge sources. It comprises two key phases:\n     - **Indexing**: Organizes documents efficiently for retrieval, utilizing inverted indexes or dense vector encoding.\n     - **Searching**: Identifies and fetches relevant documents based on user queries. This process may be supplemented by rerankers that refine the ranking of retrieved documents for better relevance.\n\n2. **Generation Component**: \n   - The generation phase utilizes the content retrieved in response to user queries to produce coherent and contextually relevant replies. It employs techniques like prompting to enhance the quality of these generated responses. During this phase, the model interprets the provided input and integrates the extracted information without needing additional fine-tuning, thereby generating accurate res

In [13]:
# Show the retrieved nodes
for src in res.source_nodes:
    print("Node ID\t", src.node_id)
    print("Title\t", src.metadata["title"])
    print("Text\t", src.text.strip())
    print("Score\t", src.score)
    print("-_" * 20)

Node ID	 1c324686-fad7-4a41-bfd3-d44f9612ca91
Title	 Evaluation of Retrieval-Augmented Generation: A Survey:Research Paper Information
Text	 Authors: Hao Yu, Aoran Gan, Kai Zhang, Shiwei Tong, Qi Liu, and Zhaofeng Liu.Numerous studies of Retrieval-Augmented Generation (RAG) systems have emerged from various perspectives since the advent of Large Language Models (LLMs). The RAG system comprises two primary components: Retrieval and Generation. The retrieval component aims to extract relevant information from various external knowledge sources. It involves two main phases, indexing and searching. Indexing organizes documents to facilitate efficient retrieval, using either inverted indexes for sparse retrieval or dense vector encoding for dense retrieval. The searching component utilizes these indexes to fetch relevant documents based on the user's query, often incorporating the optional rerankers to refine the ranking of the retrieved documents. The generation component utilizes the retr

# Custom Postprocessor


## The `Judger` Function


The following function will query GPT-4o to retrieve the top three nodes that has highest similarity to the asked question.


In [14]:
from pydantic import BaseModel
from llama_index.llms.openai import OpenAI
from llama_index.core.prompts import PromptTemplate


def judger(nodes, query):

    # The model's output template
    class OrderedNodes(BaseModel):
        """A node with the id and assigned score."""

        node_id: list
        score: list

    # Prepare the nodes and wrap them in <NODE></NODE> identifier, as well as the query
    the_nodes = ""
    for idx, item in enumerate(nodes):
        the_nodes += f"<NODE{idx+1}>\nNode ID: {item.node_id}\nText: {item.text}\n</NODE{idx+1}>\n"

    query = "<QUERY>\n{}\n</QUERY>".format(query)

    # Define the prompt template
    prompt_tmpl = PromptTemplate(
        """
    You receive a qurey along with a list of nodes' text and their ids. Your task is to assign score
    to each node based on its contextually closeness to the given query. The final output is each
    node id along with its proximity score.
    Here is the list of nodes:
    {nodes_list}

    And the following is the query:
    {user_query}

    Score each of the nodes based on their text and their relevancy to the provided query.
    The score must be a decimal number between 0 an 1 so we can rank them."""
    )

    # Define the an instance of GPT-4 and send the request
    llm = OpenAI(model="gpt-4o-mini")
    ordered_nodes = llm.structured_predict(
        OrderedNodes, prompt_tmpl, nodes_list=the_nodes, user_query=query
    )

    return ordered_nodes

## Define Postprocessor


The following class will use the `judger` function to rank the nodes, and filter them based on the ranks.


In [15]:
from typing import List, Optional
from llama_index.core import QueryBundle
from llama_index.core.postprocessor.types import BaseNodePostprocessor
from llama_index.core.schema import NodeWithScore


class OpenaiAsJudgePostprocessor(BaseNodePostprocessor):
    def _postprocess_nodes(
        self, nodes: List[NodeWithScore], query_bundle: Optional[QueryBundle]
    ) -> List[NodeWithScore]:

        r = judger(nodes, query_bundle)

        node_ids = r.node_id
        scores = r.score

        sorted_scores = sorted(enumerate(scores), key=lambda x: x[1], reverse=True)
        top_three_nodes = [sorted_scores[i][0] for i in range(3)]

        selected_nodes_id = [node_ids[item] for item in top_three_nodes]

        final_nodes = []
        for item in nodes:
            if item.node_id in selected_nodes_id:
                final_nodes.append(item)

        return final_nodes

In [16]:
judge = OpenaiAsJudgePostprocessor()

## Query Engine with Postprocessor


In [17]:
# Define a query engine that is responsible for retrieving related pieces of text,
# and using a LLM to formulate the final answer.
# The `node_postprocessors` function will be applied to the retrieved nodes.
query_engine = index.as_query_engine(similarity_top_k=5, node_postprocessors=[judge])

res = query_engine.query("Explain how Parameter Efficient Fine tuning (PEFT) works?")

In [18]:
res.response

'Parameter Efficient Fine-Tuning (PEFT) is a method designed to adapt large pretrained models efficiently for various downstream applications without the need to fine-tune all model parameters. Instead of updating all parameters, PEFT focuses on fine-tuning a small subset of additional parameters. This significantly reduces both computational and storage costs while still achieving performance that is comparable to traditional full fine-tuning methods. \n\nPEFT integrates with popular libraries, allowing for an optimized process for loading, training, and using large models for inference. This integration facilitates faster performance and makes it more feasible for users to work with large language models on consumer hardware. Additionally, PEFT methods provide user support resources, such as quick start guides and how-to guides, to assist users in applying these techniques across various tasks, enhancing accessibility and efficiency in model training and application.'

In [19]:
# Show the retrieved nodes
for src in res.source_nodes:
    print("Node ID\t", src.node_id)
    print("Title\t", src.metadata["title"])
    print("Text\t", src.text)
    print("Score\t", src.score)
    print("-_" * 20)

Node ID	 6be88fa3-2f8b-43e7-aba0-d874b39809fc
Title	 FourierFT: Discrete Fourier Transformation Fine-Tuning
Text	 # FourierFT: Discrete Fourier Transformation Fine-Tuning[FourierFT](https://huggingface.co/papers/2405.03003) is a parameter-efficient fine-tuning technique that leverages Discrete Fourier Transform to compress the model's tunable weights. This method outperforms LoRA in the GLUE benchmark and common ViT classification tasks using much less parameters.FourierFT currently has the following constraints:- Only `nn.Linear` layers are supported.- Quantized layers are not supported.If these constraints don't work for your use case, consider other methods instead.The abstract from the paper is:> Low-rank adaptation (LoRA) has recently gained much interest in fine-tuning foundation models. It effectively reduces the number of trainable parameters by incorporating low-rank matrices A and B to represent the weight change, i.e., Delta W=BA. Despite LoRA's progress, it faces storage ch